# 2b Mash filtration & clustering

In this notebook, we run Mash to generate genome-wise pairwise similarity scores (which correspond to Average Nucleotide Identify (ANI) & DNA-DNA reassociation value).

Mash will be used as a final filtration metric to filter out strains which are too dissimilar from the rest of the genome collection.

In [1]:
import subprocess
import pickle

import numpy as np
import pandas as pd
import scipy.cluster.hierarchy as hc
import scipy.spatial as sp

import matplotlib
import matplotlib.patches as patches
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px

from tqdm.notebook import tqdm

from kneebow.rotor import Rotor

# pyphyon import
import pyphylon.mash as mash

In [2]:
RAW = "data/raw/"
FNA_GENOMES = "data/raw/genomes/fna/"

In [3]:
scrubbed_species_summary = pd.read_pickle('data/interim/genome_summary_2a.pickle')
scrubbed_species_metadata = pd.read_pickle('data/interim/genome_metadata_2a.pickle')


display(
    scrubbed_species_summary.shape,
    scrubbed_species_summary.head(),
    scrubbed_species_metadata.shape,
    scrubbed_species_metadata.head()
)

(241, 20)

,genome_id,genome_name,taxon_id,genome_status,genome_length,gc_content,contig_l50,contig_n50,chromosomes,plasmids,contigs,patric_cds,refseq_cds,trna,rrnacoarse_consistency,fine_consistency,checkm_completeness,checkm_contamination,genome_qualitydate_created,date_modified
0,1010840.4,Streptococcus pyogenes MGAS1882,1010840,Complete,1781029,38.00,1,1,1,0,1,1727,0,57,NaN,99.5,100.0,0.0,NaN,2015-03-16T03:17:09.594Z
1,1048264.3,Streptococcus pyogenes HKU QMH11M0907901,1048264,Complete,1908100,38.45,1,1,1,NaN,1,1909,1865,67,NaN,99.9,100.0,0.9,NaN,2016-01-17T15:29:01.552Z
2,1150773.3,Streptococcus pyogenes JRS4,1150773,Complete,1811968,38.63,1,1,1,NaN,1,1811,1671,67,NaN,99.8,100.0,0.0,NaN,2016-01-17T16:03:54.402Z
3,1150773.4,Streptococcus pyogenes JRS4,1150773,Complete,1811124,38.64,1,1,1,NaN,1,1886,1890,66,NaN,99.9,100.0,0.0,NaN,2016-03-01T06:31:23.641Z
5,1235829.3,Streptococcus pyogenes A20,1235829,Complete,1837281,38.54,1,1,1,0,1,1834,1828,67,NaN,99.4,100.0,0.0,NaN,2015-03-16T03:17:09.594Z


(241, 66)

,genome_id,genome_name,organism_name,taxon_id,genome_status,strain,serovar,biovar,pathovar,mlst,...,motility,sporulation,temperature_range,optimal_temperature,salinity,oxygen_requirement,habitat,disease,comments,additional_metadata
0,1010840.4,Streptococcus pyogenes MGAS1882,Streptococcus pyogenes MGAS1882,1010840,Complete,MGAS1882,NaN,NaN,NaN,MLST.Streptococcus_pyogenes.172,...,NaN,NaN,NaN,NaN,NaN,NaN,Host,NaN,-,NaN
1,1048264.3,Streptococcus pyogenes HKU QMH11M0907901,NaN,1048264,Complete,HKU QMH11M0907901,NaN,NaN,NaN,MLST.Streptococcus_pyogenes.36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Clinical use of next generation sequencing for...,NaN
2,1150773.3,Streptococcus pyogenes JRS4,NaN,1150773,Complete,JRS4,serovar emm6,NaN,NaN,MLST.Streptococcus_pyogenes.37,...,NaN,NaN,NaN,C,NaN,NaN,NaN,NaN,We report the complete genome assemblies of th...,collected_by:Rockefeller University Lancefield...
3,1150773.4,Streptococcus pyogenes JRS4,NaN,1150773,Complete,JRS4,NaN,NaN,NaN,MLST.Streptococcus_pyogenes.37,...,No,NaN,NaN,NaN,NaN,NaN,NaN,Pharyngitis,Complete genome sequence of the highly invasiv...,NaN
5,1235829.3,Streptococcus pyogenes A20,Streptococcus pyogenes A20,1235829,Complete,A20,NaN,NaN,NaN,MLST.Streptococcus_pyogenes.28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Necrotizing faciitis,A clinical strain was isolated from a Necrotiz...,NaN


## Run Mash

- Running Mash on terminal:
    - `mash sketch -o data/raw/combined_sketch.msh data/raw/genomes/fna/*.fna`
- Running Mash using pyphylon:
    - Run the following cell below:

In [4]:
result = mash.sketch_genomes(genome_dir='data/raw/genomes/fna/*.fna', output_file='data/raw/combined_sketch.msh')

# Print the output and error (if any)
print("STDOUT:", result.stdout.decode())
print("STDERR:", result.stderr.decode())

CalledProcessError: Command '['mash', 'sketch', '-o', 'data/raw/combined_sketch.msh', 'data/raw/genomes/fna/*.fna']' returned non-zero exit status 127.